# Langchain: The basics

In [ ]:
!pip -q install openai langchain huggingface_hub

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.5/55.5 KB 2.5 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 282.4/282.4 KB 10.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.3/190.3 KB 19.5 MB/s eta 0:00:00


In [ ]:
import os

os.environ['OPENAI_API_KEY'] = ''
os.environ['HUGGINGFACEHUB_API_TOKEN'] = ''

## Plain Conditional Generation

### First with OpenAI GPT3 

In [ ]:
from langchain.llms import OpenAI

In [ ]:
llm = OpenAI(model_name='text-davinci-003', 
             temperature=0.9, 
             max_tokens = 256)

In [ ]:
text = "Why did the duck cross the road?"

print(llm(text))



To get to the other side.


### Now with T5-Flan-XL

In [ ]:
from langchain.llms import HuggingFaceHub

In [ ]:

llm_hf = HuggingFaceHub(
    repo_id="google/flan-t5-xl",
    model_kwargs={"temperature":0.9 }
)

In [ ]:
text = "Why did the chicken cross the road?"

print(llm_hf(text))

It was hungry.


## Prompt Templates

In [ ]:
from langchain import PromptTemplate


restaurant_template = """
I want you to act as a naming consultant for new restaurants.

Return a list of restaurant names. Each name should be short, catchy and easy to remember. It shoud relate to the type of restaurant you are naming.

What are some good names for a restaurant that is {restaurant_desription}?
"""

prompt = PromptTemplate(
    input_variables=["restaurant_desription"],
    template=restaurant_template,
)

In [ ]:
# An example prompt with one input variable
prompt_template = PromptTemplate(input_variables=["restaurant_desription"], template=restaurant_template)


In [ ]:
description = "a Greek place that serves fresh lamb souvlakis and other Greek food "
description_02 = "a burger place that is themed with baseball memorabilia"
description_03 = "a cafe that has live hard rock music and memorabilia"

## to see what the prompt will be like
prompt_template.format(restaurant_desription=description)

'\nI want you to act as a naming consultant for new restaurants.\n\nReturn a list of restaurant names. Each name should be short, catchy and easy to remember. It shoud relate to the type of restaurant you are naming.\n\nWhat are some good names for a restaurant that is a Greek place that serves fresh lamb souvlakis and other Greek food ?\n'

In [ ]:
## querying the model with the prompt template
from langchain.chains import LLMChain


chain = LLMChain(llm=llm, prompt=prompt_template)

# Run the chain only specifying the input variable.
print(chain.run(description_03))


1. Rockin' Cafe 
2. Guitar Grind 
3. Rockin' Roost 
4. Electric Brew 
5. Jammin' Java 
6. Rocker's Bistro 
7. Live & Loud Cafe 
8. Amp'd Up Cafe 
9. Rock Stop Cafe 
10. Amp'd Brews & Bites


## with Few Shot Learning

In [ ]:
from langchain import PromptTemplate, FewShotPromptTemplate


In [ ]:
# First, create the list of few shot examples.
examples = [
    {"word": "happy", "antonym": "sad"},
    {"word": "tall", "antonym": "short"},
]

In [ ]:
# Next, we specify the template to format the examples we have provided.
# We use the `PromptTemplate` class for this.
example_formatter_template = """
Word: {word}
Antonym: {antonym}\n
"""
example_prompt = PromptTemplate(
    input_variables=["word", "antonym"],
    template=example_formatter_template,
)

In [ ]:
# Finally, we create the `FewShotPromptTemplate` object.
few_shot_prompt = FewShotPromptTemplate(
    # These are the examples we want to insert into the prompt.
    examples=examples,
    # This is how we want to format the examples when we insert them into the prompt.
    example_prompt=example_prompt,
    # The prefix is some text that goes before the examples in the prompt.
    # Usually, this consists of intructions.
    prefix="Give the antonym of every input",
    # The suffix is some text that goes after the examples in the prompt.
    # Usually, this is where the user input will go
    suffix="Word: {input}\nAntonym:",
    # The input variables are the variables that the overall prompt expects.
    input_variables=["input"],
    # The example_separator is the string we will use to join the prefix, examples, and suffix together with.
    example_separator="\n\n",
)


In [ ]:

# We can now generate a prompt using the `format` method.
print(few_shot_prompt.format(input="big"))

Give the antonym of every input


Word: happy
Antonym: sad




Word: tall
Antonym: short



Word: big
Antonym:


In [ ]:
from langchain.chains import LLMChain

chain = LLMChain(llm=llm, prompt=few_shot_prompt)

# Run the chain only specifying the input variable.
print(chain.run("Big"))

 Small
